In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from tqdm import tqdm

In [2]:
from Main_function import get_Data_On_Page, get_Data_OneProduct_Link 

# URL Lazada: dienthoai, sort price high to low
path = 'chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
#Empty list, to store data from all pages
AllData = []

#Nhận link cho 4 page tiếp theo và sau đó đưa vào vòng lặp for cùng với hàm get_Data_On_Page
for page in tqdm(range(2, 6), desc="Crawling pages"):
    main_link = f'https://www.lazada.vn/catalog/?page={page}&q=laptop'
    driver.get(main_link)
    sleep(random.uniform(2,5))                             #Random sleep to mimic human behavior and avoid getting blocked

    df_Laptop = get_Data_On_Page(driver)
    AllData.append(df_Laptop)

Crawling pages: 100%|██████████| 4/4 [01:45<00:00, 26.31s/it]


In [4]:
#Combine into a DataFrame
Page02_To_Page05 = pd.concat(AllData, ignore_index=True)
Page02_To_Page05.index = np.arange(1, len(Page02_To_Page05)+1)
Page02_To_Page05

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location
1,laptop,Thành lý máy tính sách tay laptop học tập sử d...,https://www.lazada.vn/products/thanh-ly-may-ti...,864.999 ₫,28% Off,None,None,Hưng Yên
2,laptop,"Bán Xác Laptop Acer 4349, không lên hình. Giá ...",https://www.lazada.vn/products/ban-xac-laptop-...,400.000 ₫,,None,None,Tiền Giang
3,laptop,"Laptop HP 840 G3 i5 i7 Làm Văn Phòng, Học Tập ...",https://www.lazada.vn/products/laptop-hp-840-g...,3.800.000 ₫,46% Off,None,None,Hồ Chí Minh
4,laptop,[Trả góp 0%]LAPTOP CHƠI FIFA 4 PUBG MOBI PHOTO...,https://www.lazada.vn/products/tra-gop-0laptop...,2.300.000 ₫,8% Off,None,None,Hà Nội
5,laptop,Laptop hp 6470b core i5 ram 4g ổ ssd 120g giá rẻ,https://www.lazada.vn/products/laptop-hp-6470b...,1.300.000 ₫,,None,None,Hà Nội
...,...,...,...,...,...,...,...,...
156,laptop,Laptop Razer Blade 15 Advanced Model (2021) | ...,https://www.lazada.vn/products/laptop-razer-bl...,42.315.000 ₫,,7 Đã bán,None,Hồ Chí Minh
157,laptop,Laptop HP ProBook 450 G10 873J6PA (Core i5 134...,https://www.lazada.vn/products/laptop-hp-probo...,20.950.000 ₫,7% Off,None,None,Hồ Chí Minh
158,laptop,Laptop Mini GPD Pocket 3,https://www.lazada.vn/products/laptop-mini-gpd...,29.900.000 ₫,,5 Đã bán,None,Hà Nội
159,laptop,laptop 2 trong 1 kiêm máy tính bảng Fujitsu Q7...,https://www.lazada.vn/products/laptop-2-trong-...,2.500.000 ₫,,None,(1),Hưng Yên


In [5]:
#Initialize dictionary to store details for all products
Products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(Page02_To_Page05.Link, desc="Crawling link product"):
    LinkProduct = get_Data_OneProduct_Link(driver, link)
    for key in Products.keys():
        Products[key].append(LinkProduct[key])

Crawling link product: 100%|██████████| 160/160 [1:59:38<00:00, 44.87s/it] 


In [6]:
#Add columns to combined_data
for key, value in Products.items():
    Page02_To_Page05[key] = value

In [7]:
# Convert to csv
Page02_To_Page05.to_csv("Page02_To_Page05.csv", index=False)
Page02_To_Page05

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Return,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
1,laptop,Thành lý máy tính sách tay laptop học tập sử d...,https://www.lazada.vn/products/thanh-ly-may-ti...,864.999 ₫,28% Off,None,None,Hưng Yên,None,None,None,None,None,None,[1],[0],[0],[0],[1]
2,laptop,"Bán Xác Laptop Acer 4349, không lên hình. Giá ...",https://www.lazada.vn/products/ban-xac-laptop-...,400.000 ₫,,None,None,Tiền Giang,None,40.300 ₫,None,87%,37%,100%,[0],[0],[0],[0],[2]
3,laptop,"Laptop HP 840 G3 i5 i7 Làm Văn Phòng, Học Tập ...",https://www.lazada.vn/products/laptop-hp-840-g...,3.800.000 ₫,46% Off,None,None,Hồ Chí Minh,7.000.000 ₫,16.500 ₫,None,96%,80%,94%,[0],[0],[0],[1],[0]
4,laptop,[Trả góp 0%]LAPTOP CHƠI FIFA 4 PUBG MOBI PHOTO...,https://www.lazada.vn/products/tra-gop-0laptop...,2.300.000 ₫,8% Off,None,None,Hà Nội,3.000.000 ₫,104.900 ₫,None,92%,66%,85%,[0],[0],[0],[0],[1]
5,laptop,Laptop hp 6470b core i5 ram 4g ổ ssd 120g giá rẻ,https://www.lazada.vn/products/laptop-hp-6470b...,1.300.000 ₫,,None,None,Hà Nội,None,104.900 ₫,None,92%,66%,85%,[0],[0],[0],[0],[8]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,laptop,Laptop Razer Blade 15 Advanced Model (2021) | ...,https://www.lazada.vn/products/laptop-razer-bl...,42.315.000 ₫,,7 Đã bán,None,Hồ Chí Minh,None,74.700 ₫,None,92%,20%,90%,[0],[0],[0],[0],[0]
157,laptop,Laptop HP ProBook 450 G10 873J6PA (Core i5 134...,https://www.lazada.vn/products/laptop-hp-probo...,20.950.000 ₫,7% Off,None,None,Hồ Chí Minh,22.590.000 ₫,Miễn phí,30 Ngày Trả Hàng Miễn Phí,94%,86%,100%,[0],[0],[0],[0],[0]
158,laptop,Laptop Mini GPD Pocket 3,https://www.lazada.vn/products/laptop-mini-gpd...,29.900.000 ₫,,5 Đã bán,None,Hà Nội,None,34.100 ₫,None,90%,100%,75%,[0],[0],[0],[0],[0]
159,laptop,laptop 2 trong 1 kiêm máy tính bảng Fujitsu Q7...,https://www.lazada.vn/products/laptop-2-trong-...,2.500.000 ₫,,None,(1),Hưng Yên,None,80.200 ₫,None,91%,65%,100%,[0],[0],[0],[0],[1]
